In [1]:
import os
import numpy as np
import linkml_py
from linkml.processing.plane.numpy.functions import fit_plane, fit_planes
from linkml.visualize import polyscope

In [2]:
points = np.load("/home/mephisto/server_data/Amtssygehus/points.npy")
normals = np.load("/home/mephisto/server_data/Amtssygehus/normals.npy")
colors = np.load("/home/mephisto/server_data/Amtssygehus/colors.npy")

In [3]:
planes = np.load("/home/mephisto/server_data/Amtssygehus/planes.npy")
planes = planes.reshape((-1, 4))
planes = [linkml_py.Plane(plane[0], plane[1], plane[2], plane[3]) for plane in planes]

In [4]:
points_pp = []
indecies = []
for path, folders, files in os.walk("/home/mephisto/server_data/Amtssygehus/planes/"):

    # Sorting
    keys = [int(name.split(".")[0].split("-")[1]) for name in files]
    files = [x for _, x in sorted(zip(keys, files))]


    for file in files:
        if "points" in file:
            points_pp.append(np.load(os.path.join(path, file)))
        elif "indecies" in file:
            indecies.append(np.load(os.path.join(path, file)))


In [5]:
plane_results = linkml_py.PlaneFittingResults.from_numpy(planes, indecies)
plane_results

Planes :203

In [6]:
points = points*0.1

In [8]:
cloud = linkml_py.PointCloud.from_numpy(points, normals)

In [7]:
params = linkml_py.Refinement_Parameters()

In [9]:
res = linkml_py.refine_planes(cloud, plane_results, params)
print(len(res))
res

148


[Plane A(5.40554e-05) B(-0.000635126) C(1) D(-16.4968),
 Plane A(2.21168e-06) B(-4.30339e-05) C(1) D(-9.62706),
 Plane A(9.18041e-07) B(-6.20718e-05) C(1) D(-6.29311),
 Plane A(-3.17594e-06) B(-2.19511e-05) C(1) D(-2.95925),
 Plane A(0) B(0) C(1) D(-6.561),
 Plane A(0) B(0) C(1) D(-3.227),
 Plane A(-0.000949588) B(-0.005803) C(-0.999983) D(0.010541),
 Plane A(0) B(0) C(1) D(-9.895),
 Plane A(0.0204836) B(0.999396) C(-0.0280793) D(6.44947),
 Plane A(3.52619e-05) B(0.155654) C(0.987812) D(-17.5871),
 Plane A(0.0281981) B(-0.469408) C(0.882531) D(-6.14208),
 Plane A(0.0150655) B(-0.963099) C(-0.268725) D(4.43716),
 Plane A(-0.0324499) B(-0.999438) C(-0.00847873) D(1.09589),
 Plane A(-9.94558e-16) B(1) C(-1.77636e-14) D(-1.15562),
 Plane A(-0.111256) B(-0.969794) C(-0.217076) D(4.42336),
 Plane A(0.0375359) B(-0.179347) C(-0.98307) D(6.47099),
 Plane A(-0.550779) B(0.831636) C(0.0708757) D(-1.70103),
 Plane A(-0.626788) B(-0.775116) C(-0.0795714) D(1.19437),
 Plane A(0.0332463) B(-0.670059

In [ ]:
with polyscope() as ps:
    ps.set_up_dir("z_up")
    ps.set_ground_plane_mode("shadow_only")
    cloud = ps.register_point_cloud("Cloud", points=points, radius=0.002)
    cloud.add_color_quantity("RGB", colors/255, enabled=True)

In [ ]:
r = fit_planes(points, normals, distance_threshhold=2)
r

In [ ]:
# np.save("/home/mephisto/server_data/Amtssygehus/planes",np.concatenate(r))

# for idx, plane in enumerate(r):
#     np.save(f"/home/mephisto/server_data/Amtssygehus/planes/indecies-{idx}", plane.indecies)
#     np.save(f"/home/mephisto/server_data/Amtssygehus/planes/points-{idx}", plane.points)


In [ ]:
from linkml.processing.mapping import remap

In [ ]:
with polyscope() as ps:
    ps.set_up_dir("z_up")
    ps.set_ground_plane_mode("shadow_only")
    
    c = ps.register_point_cloud("Cloud", points=points, radius=0.002)
    c.add_color_quantity("RGB", colors/255, enabled=True)
    c.add_color_quantity("Normals", remap(normals), enabled=True)


    # for idx, plane in enumerate(r):
    #     ps.register_point_cloud(f"Plane {idx}", points=plane.points, radius=0.0025)


In [ ]:
import colorsys

In [ ]:
[colorsys.hsv_to_rgb(1,1,val) for val in np.random.random(10)][0]

In [ ]:
def vg_formatter(path, points, normals, colors, planes):

    with open(path, "w") as f:
        f.write(f"num_points: {len(points)}\n")
        f.writelines([f"{pt[0]:0.6f} {pt[1]:0.6f} {pt[2]:0.6f}\n" for pt in points    ])
        f.write(f"\n")

        f.write(f"num_colors: {len(colors)}\n")
        f.writelines([f"{pt[0]:0.6f} {pt[1]:0.6f} {pt[2]:0.6f}\n" for pt in colors/255])
        f.write(f"\n")

        f.write(f"num_normals: {len(normals)}\n")
        f.writelines([f"{ n[0]:0.6f} { n[1]:0.6f} { n[1]:0.6f}\n" for n  in normals   ])
        f.write(f"\n")
        f.write(f"num_groups: {len(planes)}\n")

        for idx, (plane, color) in enumerate(zip(planes, [colorsys.hsv_to_rgb(val,0.9,1) for val in np.random.random(len(planes))])):
            f.write("group_type: 0\n")
            f.write("num_group_parameters: 4\n")
            f.write(f"group_parameters: {plane[0]:0.6f} {plane[1]:0.6f} {plane[2]:0.6f} {plane[3]:0.6f}\n")
            f.write(f"group_label: plane-{idx}\n")
            f.write(f"group_color: {color[0]:0.3f} {color[1]:0.3f} {color[2]:0.3f}\n")
            f.write(f"group_num_points: {len(plane.indecies)}\n")
            f.write(" ".join([f"{val:d}" for val in plane.indecies]))
            f.write("\n")
            f.write("num_children: 0\n")

In [ ]:
vg_formatter("/home/mephisto/server_data/Amtssygehus/Point Cloud-50.vg", points, normals, colors, r[:50])

In [ ]:
cloud = linkml_py.PointCloud.from_numpy(points, normals)

In [ ]:
rs = linkml_py.fit_plane(cloud, linkml_py.PlaneFittingParams(
    cosalpha=np.cos(25),
    normal_distance_threshhold=0.1,
    distance_threshhold=2,
    plane_size_threshhold=50
))
rs

In [ ]:
with polyscope() as ps:
    ps.set_up_dir("z_up")
    ps.set_ground_plane_mode("shadow_only")
    
    c = ps.register_point_cloud("Cloud", points=points, radius=0.002)
    c.add_color_quantity("RGB", colors/255, enabled=True)

    ps.register_point_cloud("Plane", points=points[rs.indecies], radius=0.0025)

    c = ps.register_point_cloud("Point", points=points[15401:15402], radius=0.02)


    


In [ ]:
r = linkml_py.fit_planes(cloud, linkml_py.PlaneFittingParams(
    cosalpha=np.cos(25),
    normal_distance_threshhold=0.1,
    distance_threshhold=0.8,
    plane_size_threshhold=500
))
r

In [ ]:
linkml_py.PlaneFittingParams()